In [15]:
#!pip install pandas
#!pip install pymysql
#!pip install sqlalchemy pymysql

In [17]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import sqlalchemy
import numpy as np

In [19]:
# Read in file
data = pd.read_csv('survey_results_public.csv')

In [21]:
try:
    connection = pymysql.connect(host='localhost',
                                 port=3306,
                                 user='root',
                                 password='rootroot')

    # Create a cursor object
    cursor = connection.cursor()

    # Execute the CREATE DATABASE query
    cursor.execute("CREATE DATABASE stackflow")

    # Commit the changes
    connection.commit()

except Exception as e:
    print('Exception: ' + str(e))

In [23]:
username = 'root'
password = 'rootroot'
host = 'localhost'
port = 3306
database = 'stackflow'
connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'

# Create SQLAlchemy engine
engine = create_engine(connection_string)

table_name = 'so'
data.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

65437

In [25]:
# Read the current table 'SO' from the database into a DataFrame
df = pd.read_sql('SELECT * FROM so', con=engine)

# Count the number of NULLs in each row
df['null_count'] = df.isnull().sum(axis=1)

# Sort the DataFrame by 'null_count' in descending order
df_sorted = df.sort_values(by='null_count', ascending=False)

# Calculate the number of rows to drop (top 1/3 rows with the most NULL values)
num_rows_to_drop = len(df_sorted) // 3

# Drop the rows with the most NULL values
cleaned_df = df_sorted.iloc[num_rows_to_drop:]

# Drop the 'null_count' column as it's no longer needed
cleaned_df = cleaned_df.drop(columns=['null_count'])

# Print the number of rows in the cleaned DataFrame
print(f'The number of rows after cleaning: {len(cleaned_df)}')


The number of rows after cleaning: 43625


In [27]:
# Count the number of NULLs in each row
cleaned_df['null_count'] = cleaned_df.isnull().sum(axis=1)

# Sort the DataFrame by 'null_count' in descending order
df_sorted = cleaned_df.sort_values(by='null_count', ascending=False)

# Calculate the number of rows to drop (top 1/3 rows with the most NULL values)
num_rows_to_drop = len(df_sorted) // 3

# Drop the rows with the most NULL values
cleaned_df = df_sorted.iloc[num_rows_to_drop:]

# Drop the 'null_count' column as it's no longer needed
cleaned_df = cleaned_df.drop(columns=['null_count'])

# Print the number of rows in the cleaned DataFrame
print(f'The number of rows after cleaning: {len(cleaned_df)}')

The number of rows after cleaning: 29084


In [29]:
# Get the top 5 most frequent languages
languages = ['Python', 'JavaScript', 'SQL', 'TypeScript', 'HTML/CSS']

# Create dummy variables for the top 5 languages
def create_language_columns(cleaned_df, languages):
    for language in languages:
        column_name = f'LanguageSkill_{language.replace("/", "").replace(" ", "")}'
        cleaned_df[column_name] = cleaned_df['LanguageWantToWorkWith'].apply(lambda x: 1 if pd.notna(x) and language in x.split(';') else 0)
    return cleaned_df

cleaned_df = create_language_columns(cleaned_df, languages)


In [31]:
# Get the top 5 most frequent databases
databases = ['PostgreSQL', 'SQLite', 'Redis', 'MySQL', 'MongoDB']

# Create dummy variables for the top 5 databases
def create_database_columns(cleaned_df, databases):
    for database in databases:
        column_name = f'Database_{database.replace("/", "").replace(" ", "")}'
        cleaned_df[column_name] = cleaned_df['DatabaseWantToWorkWith'].apply(lambda x: 1 if pd.notna(x) and database in x.split(';') else 0)
    return cleaned_df

cleaned_df = create_database_columns(cleaned_df, databases)

In [33]:
# Get the top 5 most frequent platforms
platforms = ['Amazon Web Services (AWS)', 'Microsoft Azure', 'Google Cloud', 'Heroku', 'Firebase']

# Create dummy variables for the top 5 platforms
def create_platform_columns(cleaned_df, platforms):
    for platform in platforms:
        column_name = f'Platform_{platform.replace("/", "").replace(" ", "").replace("(", "").replace(")", "")}'
        cleaned_df[column_name] = cleaned_df['PlatformWantToWorkWith'].apply(lambda x: 1 if pd.notna(x) and platform in x.split(';') else 0)
    return cleaned_df

cleaned_df = create_platform_columns(cleaned_df, platforms)

cleaned_df.head()

,ResponseId,MainBranch,Age,Employment,RemoteWork,Check,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,Database_PostgreSQL,Database_SQLite,Database_Redis,Database_MySQL,Database_MongoDB,Platform_AmazonWebServicesAWS,Platform_MicrosoftAzure,Platform_GoogleCloud,Platform_Heroku,Platform_Firebase
5814,5815,I am a developer by profession,25-34 years old,"Employed, full-time",Remote,Apples,Hobby;School or academic work,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Books / Physical media;School (i.e., Universit...",None,...,1,0,1,0,0,0,0,1,0,0
6534,6535,I am a developer by profession,35-44 years old,"Independent contractor, freelancer, or self-em...",In-person,Apples,Hobby;Professional development or self-paced l...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;Other online ...,Technical documentation;Books;Written Tutorial...,...,0,1,0,1,1,0,0,0,0,1
30796,30797,I am a developer by profession,25-34 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Profe...,"Associate degree (A.A., A.S., etc.)",Books / Physical media;Colleague;On the job tr...,Technical documentation;Blogs;Books;Stack Over...,...,1,0,1,0,0,0,0,0,0,0
38551,38552,I am a developer by profession,35-44 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Profe...,Some college/university study without earning ...,Books / Physical media;On the job training;Oth...,Technical documentation;Blogs;Books;Written Tu...,...,1,1,0,0,0,1,0,0,0,0
7698,7699,I am a developer by profession,25-34 years old,"Independent contractor, freelancer, or self-em...",Remote,Apples,Hobby;Contribute to open-source projects;Boots...,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Other online resources (e.g., videos, blogs, f...",Technical documentation;Blogs;Written Tutorial...,...,1,1,0,0,0,0,0,0,0,0


In [35]:
# Load cleaned data into MySQL
table_name = 'so'
cleaned_df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

29084

In [37]:
print('Row count after cleaning: ' + str(len(cleaned_df)))
print('Column count after cleaning: ' + str(cleaned_df.shape[1]))

nan_count_in_PlatformWantToWorkWith = cleaned_df['PlatformWantToWorkWith'].isna().sum()
print("NaN count in column PlatformWantToWorkWith:", nan_count_in_PlatformWantToWorkWith)

nan_count_in_WebframeWantToWorkWith = cleaned_df['WebframeWantToWorkWith'].isna().sum()
print("NaN count in column WebframeWantToWorkWith:", nan_count_in_WebframeWantToWorkWith)

nan_count_in_ConvertedCompYearly = cleaned_df['ConvertedCompYearly'].isna().sum()
print("NaN count in column ConvertedCompYearly:", nan_count_in_ConvertedCompYearly)

nan_count_in_JobSat = cleaned_df['JobSat'].isna().sum()
print("NaN count in column JobSat:", nan_count_in_JobSat)

Row count after cleaning: 29084
Column count after cleaning: 129
NaN count in column PlatformWantToWorkWith: 9542
NaN count in column WebframeWantToWorkWith: 7875
NaN count in column ConvertedCompYearly: 12693
NaN count in column JobSat: 1165
